### Setup

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.schema.messages import HumanMessage, SystemMessage
import os
import openai
import tiktoken
import pinecone
from tqdm.autonotebook import tqdm
from dotenv import load_dotenv
from uuid import uuid4
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from tqdm.auto import tqdm
import random
import gradio as gr
import time

load_dotenv()

c:\Users\matte\miniconda3\envs\ml\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

### Connect to Pinecone

In [2]:
index_name = "workouts"

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENVIRONMENT")
)

active_indexes = pinecone.list_indexes()
print(active_indexes)

index = pinecone.GRPCIndex(index_name)
print(index.describe_index_stats())

['workouts']
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39}},
 'total_vector_count': 39}


### Query vectorstore

In [3]:
# initialized OpenAI's embedding model
embedding_model_name = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(
    model=embedding_model_name,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [4]:
# initialized vectorstore
text_field = "text"
index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embedding_model, text_field)

In [5]:
# check
query = "shoulder workout"
vectorstore.similarity_search(query, k=3)

[Document(page_content='Shoulder Workout A (Weeks 1, 3, 5, 7)\nExercise Sets Reps\nStanding Barbell Press 6 2 - 5\nStanding Single Arm Lateral Raise 4 6 - 9\nReverse Lateral Raise on Incline Bench 3 10 - 13\nFace Pulls 3 10 - 13\nShoulder Workout B (Weeks 2, 4, 6, 8)\nExercise Sets Reps\nSeated Barbell Press 6 2 - 5\nSeated Lateral Raise 4 6 - 9\nSingle Arm Machine Lateral Raise 3 10 - 13\nSeated Reverse Machine Fly 3 10 - 13\nMUSCLEANDSTRENGTH.COM\nTHE TOOLS YOU NEED TO BUILD \nTHE BODY YOU WANT®\nStore Workouts Diet Plans Expert Guides Videos Tools\nTHE PERFECT WORKOUT FOR ROCK SOLID \nSHOULDERS\nBuild perfectly balanced shoulders with \ndeltoids that cap with this workout routine. \nThe workout incorporates 2 workouts that you \nalternate weekly.\nLink to Workout : https://www.muscleandstrength.com/\nworkouts/perfect-workout-for-shoulders\nMain Goal:  Build Muscle\nTraining Level:  Intermediate \nProgram Duration:  8 Weeks\nDays Per Week:  1 Day\nTime Per Workout:  45-75 MinsEquipme

### Initialized LLM

In [6]:
# initialize GPT4
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4-1106-preview",
    temperature=0.0
)

### Define prompts

In [7]:
system_prompt = """
You're the world's best personal trainer.
You always provide your clients with all the information needed to become fitter, stronger and healthier through physical training.
You use your science science know and expertise, nutrition advice, and other relevant factors to create workout routines suitable to your clients.
If clients tell you they do not have access to gym equipments, you never fail to recommend exercises that do not require any tool or equipment.

For each exercise you always provide the reps, sets and rest intervals in seconds appropriate for each exercise and the client's fitness level.
You start each workout program with about 5 minutes of warm-up exercises to make the body ready for more strenuous activities and make it easier to exercise.
You end each workout routine with 5 about minutes of cool-down exercises to ease the body, lower the chance of injury, promote blood flow, and reduce stress to the heart and the muscles.
The warm-up and cool-down exercises are always different and they are always appropriate for the muscle group the person wants to train.
You never recommend exercises in the main workout routine in the warm-up or cool-down sections.
Remember, when clients tell you they do not have access to gym equipments, all the exercises you recommend, including the warm-up and cool-down exercises, can be performed without any tool.
You always limit yourself to respond with the list of exercises. You never add any additional comment.

Design the workout based on the following information:
{workout_context}


Output format:
## 🤸 Warp-up:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)

## 🏋️‍♀️ Workout
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)
...
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)

## 🧘 Cool-down:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)
""".strip()

### Generate response

In [10]:
query = "40 minutes shoulder workout dumbbells only"

# retrieve most similar workouts to the input query
similar_workouts = vectorstore.similarity_search(query, k=10)

# (optional) random sample a subset of workouts to promote diversity
similar_workouts = random.sample(similar_workouts, 10)

# join together the retrieved workouts in a single string
similar_workouts = "\n\n".join([d.page_content for d in similar_workouts])

# print(system_prompt.format(workout_context=similar_workouts))

# build input messages to feed to the LLM
messages = [
    SystemMessage(content=system_prompt.format(workout_context=similar_workouts)),
    HumanMessage(content=query)
]

# generate response
for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)

## 🤸 Warm-up:
- Arm Circles (2 minutes)
- Shoulder Rolls (2 minutes)
- Cross-body Arm Swings (1 minute)
- Plank to Downward Dog (1 minute)
- Jumping Jacks (1 minute)

## 🏋️‍♀️ Workout
- Seated Dumbbell Press (8-12 reps, 4 sets, 60 seconds rest)
- Standing Dumbbell Lateral Raise (8-12 reps, 3 sets, 60 seconds rest)
- Front Dumbbell Raise (8-12 reps, 3 sets, 60 seconds rest)
- Bent-Over Dumbbell Reverse Fly (8-12 reps, 3 sets, 60 seconds rest)
- Dumbbell Shrugs (8-12 reps, 3 sets, 60 seconds rest)
- Alternating Dumbbell Front Raise (8-12 reps, 2 sets, 60 seconds rest)
- Dumbbell Arnold Press (8-12 reps, 3 sets, 60 seconds rest)

## 🧘 Cool-down:
- Chest Stretch (1 minute)
- Shoulder Stretch (1 minute)
- Triceps Stretch (1 minute)
- Cross-body Shoulder Stretch (1 minute each side)
- Standing Forward Bend (1 minute)

### Gradio demo

In [11]:
embedding_model_name = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(
    model=embedding_model_name,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4-1106-preview",
    # model_name="gpt-3.5-turbo-1106",
    temperature=0.0
)

index_name = "workouts"

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENVIRONMENT")
)

text_field = "text"
index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embedding_model, text_field)

In [12]:
system_prompt = """
You're the world's best personal trainer.
You always provide your clients with all the information needed to become fitter, stronger and healthier through physical training.
You use your science science know and expertise, nutrition advice, and other relevant factors to create workout routines suitable to your clients.
If clients tell you they do not have access to gym equipments, you never fail to recommend exercises that do not require any tool or equipment.

For each exercise you always provide the reps, sets and rest intervals in seconds appropriate for each exercise and the client's fitness level.
You start each workout program with about 5 minutes of warm-up exercises to make the body ready for more strenuous activities and make it easier to exercise.
You end each workout routine with 5 about minutes of cool-down exercises to ease the body, lower the chance of injury, promote blood flow, and reduce stress to the heart and the muscles.
The warm-up and cool-down exercises are always different and they are always appropriate for the muscle group the person wants to train.
You never recommend exercises in the main workout routine in the warm-up or cool-down sections.
Remember, when clients tell you they do not have access to gym equipments, all the exercises you recommend, including the warm-up and cool-down exercises, can be performed without any tool.
You always limit yourself to respond with the list of exercises. You never add any additional comment.

Design the workout based on the following information:
{workout_context}


Output format:
## 🤸 Warp-up:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)

## 🏋️‍♀️ Workout
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)
...
- <exercise name> (<reps> reps, <sets> sets, <rest interval> seconds rest)

## 🧘 Cool-down:
- <exercise name> (<duration> minutes)
...
- <exercise name> (<duration> minutes)
""".strip()

In [13]:
def retrieve_knowledge(query, k=10, randomize=True):
    knowledge = [d.page_content.strip() for d in vectorstore.similarity_search(query, k=k)]
    
    if randomize:
        knowledge = random.sample(knowledge, k)

    knowledge = "\n\n\n".join(knowledge)

    return knowledge


def generate_workout(query, knowledge):
    messages = [
        SystemMessage(content=system_prompt.format(workout_context=knowledge)),
        HumanMessage(content=query)
    ]

    response = llm.invoke(messages).content.strip()

    return response

In [14]:
query = "40 minutes shoulder workout dumbbells only"
workout_context = retrieve_knowledge(query, k=10, randomize=True)

response = generate_workout(query, workout_context)
print(response)

## 🤸 Warm-up:
- Arm Circles (2 minutes)
- Shoulder Shrugs (1 minute)
- Standing Wall Push (1 minute)
- Cross-body Shoulder Stretch (30 seconds each side)
- Overhead Reach and Pull (1 minute)

## 🏋️‍♀️ Workout
- Standing Dumbbell Press (6 reps, 4 sets, 90 seconds rest)
- Alternating Dumbbell Front Raise (8 reps, 3 sets, 60 seconds rest)
- Dumbbell Lateral Raise (10 reps, 3 sets, 60 seconds rest)
- Bent Over Dumbbell Reverse Fly (12 reps, 3 sets, 60 seconds rest)
- Dumbbell Shrugs (15 reps, 3 sets, 60 seconds rest)
- Standing Dumbbell Upright Row (8 reps, 3 sets, 60 seconds rest)
- Dumbbell Arnold Press (10 reps, 3 sets, 90 seconds rest)

## 🧘 Cool-down:
- Shoulder Stretch with Towel (2 minutes each side)
- Chest Stretch against Wall (1 minute each side)
- Child's Pose with Arm Extension (2 minutes)
- Neck Rolls (1 minute)


In [15]:
def run(gender, muscle_group, equipment, level, duration, k=5, randomize=True):
    query = f"{duration}-minute {muscle_group} workout for {gender} {level} level {equipment}"
    knowledge = retrieve_knowledge(query, k, randomize)
    response = generate_workout(query, knowledge)

    for i in range(len(response)):
        time.sleep(0.002)
        yield response[:i+1]

In [ ]:
css = """
#gen-button {
    background-color: #cc6600;
    color: white;
    font-size: 24px !important;
}
""".strip()

with gr.Blocks(theme=gr.themes.Monochrome(radius_size=gr.themes.sizes.radius_sm), css=css) as demo:
    with gr.Row():
        gr.Markdown("# Workout Generator")

    with gr.Row():
        with gr.Column(scale=1):
            with gr.Row():
                gender = gr.Radio(["Male", "Female"], label="Gender", elem_id="#my-button")
            with gr.Row():
                level = gr.Radio(["Beginner", "Intermediate", "Advanced"], label="Level")
            with gr.Row():
                muscle_group = gr.Radio(["Shoulders", "Chest", "Back", "Abs", "Arms", "Legs"], label="Muscle Group")
            with gr.Row():
                equipment = gr.Radio(["Gym Equipment", "Dumbbells Only", "No Equipment"], label="Equipment")
            with gr.Row():
                duration = gr.Slider(20, 60, step=5, label="Duration (minutes)")
            with gr.Row():
                # clear_button = gr.ClearButton(value="Clear Inputs")
                generate_button = gr.Button("Generate Workout", variant="primary", elem_id="gen-button")
        with gr.Column(scale=1, min_width=800, elem_id="#gen-output"):
            generation = gr.Markdown(value="")

    generate_button.click(run, inputs=[gender, level, muscle_group, equipment, duration], outputs=generation)
    # clear_button.click(fn=lambda: [None, None, None, None, None], outputs=[gender, level, muscle_group, equipment, duration])

demo.launch(share=True)